# Input size : 5 x 5 x 5
- board size : 5 x 5 
- P1 unique pieces : 2 
- P2 unique pieces : 2
- Colour

# Output size 5 x 5 x 13
- board size : 5 x 5 
- 13 possibles moves, 1 for each directions in ["N", "NE", "E", "SE", "S", "SW", "W", "NW"], plus the dragon, crab and tiger differents moves

### Layer codes

In [25]:
codes, i = {}, 0 
for nSquares in range(1,2):
    for direction in ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]:
        codes[(nSquares, direction)] = i
        i += 1
        
for direction in ["E", "W", "S"]:
    codes[(2, direction)] = i
    i += 1
    
# codes[(2, "E")] = 8
# codes[(2, "W")] = 9
# codes[(2, "S")] = 10
codes[("2E", "1S")] = 11
codes[("2W", "1S")] = 12

In [26]:
codes

{(1, 'N'): 0,
 (1, 'NE'): 1,
 (1, 'E'): 2,
 (1, 'SE'): 3,
 (1, 'S'): 4,
 (1, 'SW'): 5,
 (1, 'W'): 6,
 (1, 'NW'): 7,
 (2, 'E'): 8,
 (2, 'W'): 9,
 (2, 'S'): 10,
 ('tiger', '2E', '1S'): 11,
 ('tiger', '2W', '1S'): 12}

In [3]:
class NNet():
    
    def __init__(self):
        pass